[20220918]

# 1. Preparing

prepare data for LSTM

## Goal
To check a summary of the data.

## Plan
Checklist what will be done on the notebook :

        [ ] get the data
        [ ] generate input ready dataset

In [1]:
# !pip install flake8 pycodestyle_magic

In [2]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
sys.path.insert(1,'../../src/')

## import myfunc inside src folder
import ramal
import joblib

In [3]:
with open("../../config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

full_path = globals()['_dh'][0]
folder_name = full_path.split('/')[-1]

PATH_RAW = config['path']['raw_wm']
PATH_INTERIM = config['path']['interim']
PATH_PROCESSED = config['path']['processed']
PATH_MODEL = config['path']['artifact']+folder_name+'/'
PATH_UDF = config['path']['udf']

In [4]:
%load_ext pycodestyle_magic

In [11]:
# use latest 12 weeks data as oot
oot_n = 12

## Get Data

In [8]:
# %%flake8
df_all = pd.read_csv(PATH_INTERIM+'data_focus_iter3.csv')
df_all['is_holiday'] = df_all.IsHoliday.astype(int)
df_all.sample(8)

,ids,Date,sales,Store,Temperature,IsHoliday,Fuel_Price,sales_ss,is_holiday
450241,7:52,2011-07-29,1113.39,7,56.66,False,3.547,-0.424422,0
155590,23:16,2010-03-19,15401.41,23,43.01,False,2.834,-0.374269,0
132005,20:94,2010-05-28,75326.73,20,70.97,False,2.899,1.372422,0
303002,35:83,2012-07-20,1926.82,35,78.64,False,3.567,-0.368536,0
326783,38:42,2010-08-20,49.63,38,92.81,False,3.041,-1.806033,0
333551,39:22,2011-07-15,13368.76,39,85.96,False,3.575,0.104957,0
150266,22:55,2012-04-27,6170.84,22,51.42,False,4.023,-0.557297,0
99516,19:1,2012-08-10,14837.51,19,74.45,False,3.863,-0.458486,0


In [9]:
# %%flake8
def ts_lookback_features(df, window_back, target_cols=None, oot_data=None,
                         time_col='Date', feature_cols=['sales_ss']):

    if target_cols is None:
        target_cols = feature_cols

    # print(target_cols)
    df = df.sort_values(time_col).reset_index(drop=True)
    df['Xprev'] = df.apply(lambda x: df[feature_cols][
                    max(-1, x.name-(window_back)):int(x.name)].values, axis=1)
    df_temp = df[window_back:]
    df_temp['Xprev'] = df_temp['Xprev'].apply(lambda x: x.reshape(1,
                                              window_back, len(feature_cols)))

    X_temp = np.vstack(df_temp['Xprev'].values)
    y_temp = df_temp[target_cols].values.reshape((len(df_temp),
                                                  len(target_cols)))
    if oot_data is None:
        X_tr = X_temp[:]
        y_tr = y_temp[:]
        X_oot = []
        y_oot = []
    else:
        X_tr = X_temp[:-oot_data]
        y_tr = y_temp[:-oot_data]
        X_oot = X_temp[-oot_data:]
        y_oot = y_temp[-oot_data:]

    return X_tr, y_tr, X_oot, y_oot

## Generate 1

Sales only (lookback 5)

In [12]:
# s_id1 = '9:79'
# wb = 4
# df_temp = df_all[df_all.ids == s_id1]
# X, y =ts_lookback_features(df_temp, window_back=wb)
# X.shape

In [13]:
# %%flake8
X_all = []
y_all = []
X_oot = []
y_oot = []
wb = 5
for sid in df_all['ids'].unique():
    df_temp = df_all[df_all.ids == sid]
    Xy = ts_lookback_features(df_temp,
                              window_back=wb,
                              feature_cols=['sales_ss'],
                              oot_data=oot_n)
    X_temp, y_temp, X_oot_temp, y_oot_temp = Xy
    X_all.append(X_temp)
    y_all.append(y_temp)
    X_oot.append(X_oot_temp)
    y_oot.append(y_oot_temp)

X_all = np.vstack(X_all)
y_all = np.vstack(y_all)
X_oot = np.vstack(X_oot)
y_oot = np.vstack(y_oot)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [14]:
joblib.dump((X_all,y_all, X_oot, y_oot),PATH_INTERIM+'feat_target_gen1_iter3.pkl')

['../../data/interim/feat_target_gen1_iter3.pkl']

## Generate 2

Sales only (lookback 10)

In [15]:
X_all = []
y_all = []
X_oot = []
y_oot = []
wb = 10
for sid in df_all['ids'].unique():
    df_temp = df_all[df_all.ids == sid]
    Xy = ts_lookback_features(df_temp, window_back=wb, 
                              feature_cols=['sales_ss'],
                              oot_data = oot_n)

    X_temp, y_temp, X_oot_temp, y_oot_temp = Xy
    
    X_all.append(X_temp)
    y_all.append(y_temp)
    X_oot.append(X_oot_temp)
    y_oot.append(y_oot_temp)
    
X_all = np.vstack(X_all)
y_all = np.vstack(y_all)
X_oot = np.vstack(X_oot)
y_oot = np.vstack(y_oot)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [16]:
joblib.dump((X_all,y_all, X_oot, y_oot),PATH_INTERIM+'feat_target_gen2_iter3.pkl')

['../../data/interim/feat_target_gen2_iter3.pkl']

## Generate 3

Sales only (lookback 15)

In [17]:
X_all = []
y_all = []
X_oot = []
y_oot = []
wb = 15
for sid in df_all['ids'].unique():
    df_temp = df_all[df_all.ids == sid]
    X_temp, y_temp, X_oot_temp, y_oot_temp = ts_lookback_features(df_temp, window_back=wb, 
                                          feature_cols=['sales_ss'],
                                          oot_data = oot_n)
    X_all.append(X_temp)
    y_all.append(y_temp)
    X_oot.append(X_oot_temp)
    y_oot.append(y_oot_temp)
    
X_all = np.vstack(X_all)
y_all = np.vstack(y_all)
X_oot = np.vstack(X_oot)
y_oot = np.vstack(y_oot)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [18]:
joblib.dump((X_all,y_all, X_oot, y_oot),PATH_INTERIM+'feat_target_gen3_iter3.pkl')

['../../data/interim/feat_target_gen3_iter3.pkl']

## Generate 4

Sales and Temperature (lookback 15)

In [19]:
X_all = []
y_all = []
X_oot = []
y_oot = []
wb = 15
for sid in df_all['ids'].unique():
    df_temp = df_all[df_all.ids == sid]
    X_temp, y_temp, X_oot_temp, y_oot_temp = ts_lookback_features(df_temp, window_back=wb, 
                                          feature_cols=['sales_ss','Temperature'], 
                                          target_cols = ['sales_ss'],
                                          oot_data = oot_n)
    X_all.append(X_temp)
    y_all.append(y_temp)
    X_oot.append(X_oot_temp)
    y_oot.append(y_oot_temp)
    
X_all = np.vstack(X_all)
y_all = np.vstack(y_all)
X_oot = np.vstack(X_oot)
y_oot = np.vstack(y_oot)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [20]:
joblib.dump((X_all,y_all, X_oot, y_oot),PATH_INTERIM+'feat_target_gen4_iter3.pkl')

['../../data/interim/feat_target_gen4_iter3.pkl']

## Generate 5

Sales, Temperature, and is_holiday  (lookback 15)

In [21]:
X_all = []
y_all = []
X_oot = []
y_oot = []
wb = 15
for sid in df_all['ids'].unique():
    df_temp = df_all[df_all.ids == sid]
    X_temp, y_temp, X_oot_temp, y_oot_temp = ts_lookback_features(df_temp, window_back=wb, 
                                          feature_cols=['sales_ss','Temperature','is_holiday'], 
                                          target_cols = ['sales_ss'],
                                          oot_data = oot_n)
    X_all.append(X_temp)
    y_all.append(y_temp)
    X_oot.append(X_oot_temp)
    y_oot.append(y_oot_temp)
    
X_all = np.vstack(X_all)
y_all = np.vstack(y_all)
X_oot = np.vstack(X_oot)
y_oot = np.vstack(y_oot)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [22]:
joblib.dump((X_all,y_all, X_oot, y_oot),PATH_INTERIM+'feat_target_gen5_iter3.pkl')

['../../data/interim/feat_target_gen5_iter3.pkl']

## Generate 6

Sales, Temperature, and is_holiday  (lookback 15) + current temp and is_holiday

In [23]:
X_all = []
X_all2 = []
y_all = []
X_oot = []
X_oot2 = []
y_oot = []
wb = 15
for sid in df_all['ids'].unique():
    df_temp = df_all[df_all.ids == sid]
    X_temp, y_temp, X_oot_temp, y_oot_temp = ts_lookback_features(df_temp, window_back=wb, 
                                          feature_cols=['sales_ss','Temperature','is_holiday'], 
                                          target_cols = ['sales_ss'],
                                          oot_data = oot_n)
    X_all.append(X_temp)
    y_all.append(y_temp)
    
    X_oot.append(X_oot_temp)
    y_oot.append(y_oot_temp)
    
    
    X2 = df_temp[wb:][['Temperature','is_holiday']].values
    X_all2.append(X2[:-oot_n])
    X_oot2.append(X2[-oot_n:])
    
X_all = np.vstack(X_all)
X_all2 = np.vstack(X_all2)
y_all = np.vstack(y_all)
X_oot = np.vstack(X_oot)
X_oot2 = np.vstack(X_oot2)
y_oot = np.vstack(y_oot)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [24]:
X_all.shape

(386396, 15, 3)

In [25]:
X_all2.shape

(386396, 2)

In [26]:
X_oot.shape

(39972, 15, 3)

In [27]:
X_oot2.shape

(39972, 2)

In [28]:
joblib.dump((X_all,X_all2,y_all, X_oot,X_oot2, y_oot),PATH_INTERIM+'feat_target_gen6_iter3.pkl')

['../../data/interim/feat_target_gen6_iter3.pkl']